In [1]:
# !pip3 install gspread
# !pip3 install --upgrade google-api-python-client oauth2client 

In [2]:
quests 
import json
import gspread
import os
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
def date_to_url(s):
    """ input 17/05/1996 
        return 19962F052F17 (url format)"""
    l = s.split('/')
    if len(l)==2:
        year=2022
        day,month = l
    elif len(l)==3:
        day,month,year = l

    return f'{year}%2F{month}%2F{day}'
def time_to_int(date):
    return int(date.split(':')[0])

In [4]:
os.path.abspath('istheretgvmax-5273da7c9b7a.json')


'/home/gauffrea/Projects/is-there-tgvmax/istheretgvmax-5273da7c9b7a.json'

# 1. Google Sheet scrapping

In [5]:
gc = gspread.service_account(filename=os.path.abspath('istheretgvmax-5273da7c9b7a.json')) #Put the path of your own json google token
sh = gc.open("is-there-tgvmax")
worksheet_list = [sh for sh in sh.worksheets() if '@' in sh.title]
sh=worksheet_list[0]
origine,destination,date,heure_min,heure_max,_,email_envoye,force_update=sh.row_values(2)
print(origine,destination,date,heure_min,heure_max,_,email_envoye,force_update)

PARIS+(intramuros) GRENOBLE 22/06 17 21  FALSE FALSE


# 2. API Request

In [6]:
origine='PARIS+(intramuros)'
destination='GRENOBLE'
date='22/06'
heure_min = '17'
heure_max = '21'

In [7]:
request_start = 'https://ressources.data.sncf.com/api/records/1.0/search/?dataset=tgvmax&q=&sort=-date&facet=date&facet=origine&facet=destination&facet=od_happy_card'
full_request = f'{request_start}&refine.destination={destination}&refine.origine={origine}&refine.date={date_to_url(date)}'
request=requests.get(full_request)

In [8]:
print(full_request)

https://ressources.data.sncf.com/api/records/1.0/search/?dataset=tgvmax&q=&sort=-date&facet=date&facet=origine&facet=destination&facet=od_happy_card&refine.destination=GRENOBLE&refine.origine=PARIS+(intramuros)&refine.date=2022%2F06%2F22


In [9]:
request.text

'{"nhits": 6, "parameters": {"dataset": ["tgvmax"], "rows": 10, "start": 0, "sort": ["-date"], "facet": ["date", "origine", "destination", "od_happy_card"], "format": "json", "timezone": "UTC"}, "records": [{"datasetid": "tgvmax", "recordid": "6400fcb4cee7348e9beeebc234e746b0ca6f924b", "fields": {"heure_arrivee": "13:42", "code_equip": "TGF", "date": "2022-06-22", "train_no": "6311", "destination": "GRENOBLE", "origine": "PARIS (intramuros)", "entity": "PAGRENOBLE", "origine_iata": "FRPLY", "od_happy_card": "NON", "axe": "SUD-EST", "destination_iata": "FRGNB", "heure_depart": "9:53"}, "record_timestamp": "2022-06-16T05:30:01.413Z"}, {"datasetid": "tgvmax", "recordid": "b8cfbff3eb0885a7a783ecb293b5b4490c039900", "fields": {"heure_arrivee": "10:48", "code_equip": "TGF", "date": "2022-06-22", "train_no": "6905", "destination": "GRENOBLE", "origine": "PARIS (intramuros)", "entity": "PAGRENOBLE", "origine_iata": "FRPLY", "od_happy_card": "NON", "axe": "SUD-EST", "destination_iata": "FRGNB",

In [10]:
dico = json.loads(request.text)
voyages = dico['records']

In [11]:
for (i,voyage) in enumerate(voyages) :
    heure_depart = time_to_int(voyage['fields']['heure_depart'])
    if heure_depart>=int(heure_min) and heure_depart<=int(heure_max):
        print(i,voyage)


4 {'datasetid': 'tgvmax', 'recordid': '0a70c9698f40a395d36eab887e8c43e24ef92a10', 'fields': {'heure_arrivee': '21:42', 'code_equip': 'TGF', 'date': '2022-06-22', 'train_no': '6925', 'destination': 'GRENOBLE', 'origine': 'PARIS (intramuros)', 'entity': 'PAGRENOBLE', 'origine_iata': 'FRPLY', 'od_happy_card': 'NON', 'axe': 'SUD-EST', 'destination_iata': 'FRGNB', 'heure_depart': '18:42'}, 'record_timestamp': '2022-06-16T05:30:01.413Z'}
5 {'datasetid': 'tgvmax', 'recordid': '7be9557ad920d7c177756441565bae06889abf26', 'fields': {'heure_arrivee': '20:42', 'code_equip': 'TGF', 'date': '2022-06-22', 'train_no': '6923', 'destination': 'GRENOBLE', 'origine': 'PARIS (intramuros)', 'entity': 'PAGRENOBLE', 'origine_iata': 'FRPLY', 'od_happy_card': 'NON', 'axe': 'SUD-EST', 'destination_iata': 'FRGNB', 'heure_depart': '17:41'}, 'record_timestamp': '2022-06-16T05:30:01.413Z'}
